<a href="https://colab.research.google.com/github/MADHUSRIJ/DataTraining/blob/main/Covid19Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!apt-get install openjdk-8-jdk-headless -qq

Selecting previously unselected package libxtst6:amd64.
(Reading database ... 123105 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headless_8u372-ga~us1-0ubuntu1~20.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u372-ga~us1-0ubuntu1~20.04) ...
Selecting previously unselected package openjdk-8-jdk-headless:amd64.
Preparing to unpack .../openjdk-8-jdk-headless_8u372-ga~us1-0ubuntu1~20.04_amd64.deb ...
Unpacking openjdk-8-jdk-headless:amd64 (8u372-ga~us1-0ubuntu1~20.04) ...
Setting up libxtst6:amd64 (2:1.2.3-1) ...
Setting up openjdk-8-jre-headless:amd64 (8u372-ga~us1-0ubuntu1~20.04) ...
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/orbd to provide /usr/bin/orbd (orbd) in auto mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/ser

In [7]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=de0f2a882f12111381759a6bcef939dc0b24bfb954a79f0f3312d98c1ab8ee2b
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [9]:
from pyspark.sql import SparkSession

file_path = "/covid.csv"

spark = SparkSession.builder.getOrCreate()

df = spark.read.csv(file_path, header=True, inferSchema=True)

df.show()

+----------+------------------+--------+---------+---------------------+-----+-------------------------+---------+----------+-------------+
|      Date|Name of State / UT|Latitude|Longitude|Total Confirmed cases|Death|Cured/Discharged/Migrated|New cases|New deaths|New recovered|
+----------+------------------+--------+---------+---------------------+-----+-------------------------+---------+----------+-------------+
|2020-01-30|            Kerala| 10.8505|  76.2711|                  1.0|    0|                      0.0|        0|         0|            0|
|2020-01-31|            Kerala| 10.8505|  76.2711|                  1.0|    0|                      0.0|        0|         0|            0|
|2020-02-01|            Kerala| 10.8505|  76.2711|                  2.0|    0|                      0.0|        1|         0|            0|
|2020-02-02|            Kerala| 10.8505|  76.2711|                  3.0|    0|                      0.0|        1|         0|            0|
|2020-02-03|        

In [21]:
from pyspark.sql.functions import col, lower, month, when

# Convert state names to lowercase
df = df.withColumn("Name of State / UT", lower(col("Name of State / UT")))
df.show()

+----------+------------------+--------+---------+---------------------+-----+-------------------------+---------+----------+-------------+--------------------+-----+
|      Date|Name of State / UT|Latitude|Longitude|Total Confirmed cases|Death|Cured/Discharged/Migrated|New cases|New deaths|New recovered|death_to_cases_ratio|month|
+----------+------------------+--------+---------+---------------------+-----+-------------------------+---------+----------+-------------+--------------------+-----+
|2020-01-30|            kerala| 10.8505|  76.2711|                  1.0|    0|                      0.0|        0|         0|            0|                 0.0|    1|
|2020-01-31|            kerala| 10.8505|  76.2711|                  1.0|    0|                      0.0|        0|         0|            0|                 0.0|    1|
|2020-02-01|            kerala| 10.8505|  76.2711|                  2.0|    0|                      0.0|        1|         0|            0|                 0.0|    2

In [13]:
# Find the day with the greatest number of COVID cases
max_cases_day = df.select("Date", "New cases").orderBy(col("New cases").desc()).first()["Date"]

max_cases_day

datetime.date(2020, 7, 26)

In [14]:
# Find the state with the second-largest number of COVID cases
second_largest_cases_state = df.groupBy("Name of State / UT").sum("Total Confirmed cases").orderBy(col("sum(Total Confirmed cases)").desc()).take(2)[-1]["Name of State / UT"]

second_largest_cases_state

'tamil nadu'

In [19]:
# Cast the "Death" column to a numeric data type
df = df.withColumn("Death", df["Death"].cast("int"))

# Find the Union Territory with the least number of deaths
territory_least_deaths = df.filter(col("Death") > 0).filter(col("Name of State / UT").contains("union territory")).groupBy("Name of State / UT").sum("Death").orderBy(col("sum(Death)")).first()["Name of State / UT"]

territory_least_deaths

TypeError: ignored

In [18]:
# Find the state with the lowest Death to Total Confirmed cases ratio
df = df.withColumn("death_to_cases_ratio", col("Death") / col("Total Confirmed cases"))

lowest_death_ratio_state = df.groupBy("Name of State / UT").avg("death_to_cases_ratio").orderBy(col("avg(death_to_cases_ratio)")).first()["Name of State / UT"]

lowest_death_ratio_state

'union territory of ladakh'

In [20]:
# Find the month with the most recent recovered cases
df = df.withColumn("month", month(col("Date")))

month_with_newer_recovered_cases = df.groupBy("month").sum("New recovered").orderBy(col("sum(New recovered)").desc()).first()["month"]

month_with_newer_recovered_cases

7

In [22]:
# Convert month number to month name if it's "02"
month_with_newer_recovered_cases = when(month_with_newer_recovered_cases == 2, "February").otherwise(month_with_newer_recovered_cases)

month_with_newer_recovered_cases

PySparkTypeError: ignored